In [5]:
import numpy as np
import pandas as pd

### Load parameter

In [49]:
sheet = "Problem 3 Instance 1"
file = "OR112-2_hw02_data.xlsx"

m = pd.read_excel(file,usecols="B:B",nrows=1,sheet_name=sheet)
m = int(m.columns.item())
n = pd.read_excel(file,usecols="B:B",skiprows=range(1),nrows=1,sheet_name=sheet)
n = int(n.columns.item())
p = pd.read_excel(file,usecols="B:B",skiprows=range(2),nrows=1,sheet_name=sheet)
p = int(p.columns.item())

d = pd.read_excel(file,usecols="E:"+chr(ord('E')+n-1),skiprows=range(4),nrows=m,sheet_name=sheet)
d = d.to_numpy()

h = pd.read_excel(file,usecols="B:B",skiprows=range(4),nrows=m,sheet_name=sheet)
h = h.to_numpy()

### Variables

In [47]:
x = np.ones(n)
w = [np.min(d[i]) for i in range(m)]

total = sum(h[i]*w[i] for i in range(m)).item()


### Algorithm

In [45]:
def objfunc(h,d,x):
    total = 0
    for i in range(h.shape[0]):
        total += h[i] * min(d[i,x.astype(bool)])
    return total.item()

def column_sum(h,d,x):
    column_sums = np.zeros(x.shape[0])
    for i in range(h.shape[0]):
        column_sums[np.argmin(d[i,x.astype(bool)])] += (h[i] * min(d[i,x.astype(bool)])).item()
    return column_sums



In [51]:
x = np.ones(n)
for i in range(n-p):
    x[np.argmin(column_sum(h,d,x))] = 0
print(objfunc(h,d,x))
print(x)

17423
[6588.  780. 2244. 3128. 4683.]
28057
[1. 0. 1. 1. 0.]
